In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
from osgeo import osr

import fiona
import rasterio
import geopandas

from matplotlib import pyplot as plt

import shapely
import shapely.wkt
import shapely.geometry

import numpy as np
import math
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib
import logging

from prof import Prof

gdal.UseExceptions()

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)
        
DEBUG=False
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.ERROR)
logging.getLogger('fiona.ogrext').disabled=True # Too noisy
fiona.Env(CPL_DEBUG=DEBUG).__enter__()
rasterio.Env(CPL_DEBUG=DEBUG).__enter__()

# Lecture de l'ENC

In [ ]:
enc_path = 'data/ENC_ROOT/US4MA13M/US4MA13M.000'
m_covr_lyr = fiona.open(enc_path, layer='M_COVR', LIST_AS_STRING=True)

covr_geo = next(f for f in m_covr_lyr if f['properties']['CATCOV'] == 1)['geometry']
repr(covr_geo)[:200]

## Écriture de l'emprise dans `data/covr.geojson`

In [ ]:
covr_path = 'data/covr.geojson'
with fiona.open(covr_path, mode='w', driver='GeoJSON', schema={'properties':{}, 'geometry': 'Polygon'}) as cover_lyr:
    cover_lyr.write({'properties':{}, 'geometry': covr_geo})

# Lecture de la Bathy

## Réduction à la zone de l'ENC

In [ ]:
bathy_ds = gdal.Warp('data/bathy.tif', 'data/navd_bath_30m',
                        options = gdal.WarpOptions(
                                            format='GTiff', 
                                            cutlineDSName=covr_path,
                                            cutlineLayer='covr',
                                            cropToCutline=True,
                                            creationOptions=['COMPRESS=LZW'],
                                            dstNodata=np.nan))
del bathy_ds # ensure release and flush

In [ ]:
bathy_ds = rasterio.open('data/bathy.tif')

In [ ]:
assert len(bathy_ds.indexes) == 1
bathy_np = bathy_ds.read()[0]
bathy_ds.nodata

In [ ]:
bathy_np.shape # Northing, Easting

# Lecture de l'ENC (suite)

In [ ]:
soundg_lyr = geopandas.read_file(enc_path, layer='SOUNDG', LIST_AS_STRING=True, SPLIT_MULTIPOINT=True, ADD_SOUNDG_DEPTH=True).to_crs(bathy_ds.crs)
depare_lyr = geopandas.read_file(enc_path, layer='DEPARE', LIST_AS_STRING=True).to_crs(bathy_ds.crs)

In [ ]:
soundg_lyr

# Accès aux pixels

In [ ]:
bathy_ds.xy(0,0), bathy_ds.xy(2000,0), bathy_ds.xy(0, 2000)

In [ ]:
bathy_ds.index(328133.590626, 4722220.06), bathy_ds.index(388133.590626, 4722220.06), bathy_ds.index(328133.590626, 4662220.06)

In [ ]:
bathy_np[bathy_ds.index(415283.0,4717460.3)] # -125,9262

# Parcours d'isobathe

In [ ]:
depare_lyr

In [ ]:
" ".join(list(depare_lyr.columns))

In [ ]:
depths = - np.sort(depare_lyr.DRVAL1.unique())
depths

In [ ]:
bathy2_np = bathy_np.copy()
bathy2_np[bathy2_np>0] = np.nan

eps = .1

for d in depths:
    bathy2_np[(bathy2_np>=(d-eps)) & (bathy2_np<=(d+eps))] = 1

plt.imshow(bathy2_np)

In [ ]:
with rasterio.open(
    'test.tif',
    'w',
    driver='GTiff',
    height=bathy2_np.shape[0],
    width=bathy2_np.shape[1],
    count=1,
    dtype=bathy2_np.dtype,
    crs=bathy_ds.crs,
    transform=bathy_ds.transform,
) as dst:
    dst.write(bathy2_np, 1)


In [ ]:
depare_lyr.plot(column='DRVAL1', legend=True)

In [ ]:
dep0 = depare_lyr[depare_lyr.DRVAL1 == 0] # 18.2

In [ ]:
f=dep0.iloc[0]
f

In [ ]:
s=f.geometry.boundary

In [ ]:
f.geometry

In [ ]:
shapely.geometry.Polygon([s.interpolate(p) for p in range(0, int(s.length),1)])

In [ ]:
def f(t):
    q = bathy_ds.index(t.x,t.y)
    return (q[1]*100., -q[0]*100.)

with prof.Prof():
    r=shapely.geometry.Polygon([f(s.interpolate(p)) for p in range(0, int(s.length),30)])

In [ ]:
def f(t):
    q = bathy_ds.index(t.x,t.y)
    return q

pixels=np.array([f(s.interpolate(p)) for p in range(0, int(s.length),30)])

In [ ]:
pixels[:,0].min()

In [ ]:
for i in pixels:
    bathy_np[tuple(i)]=0

In [ ]:
bathy_np[bathy_np>0]=np.nan

In [ ]:
MARGIN=10
plt.imshow(bathy_np[pixels[:,0].min()-MARGIN:pixels[:,0].max()+MARGIN+1,pixels[:,1].min()-MARGIN:pixels[:,1].max()+MARGIN+1])